In [ ]:
%pwd

In [ ]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
from datetime import datetime
from decimal import Decimal

import pandas as pd # type: ignore
import numpy as np

import fetch_data # type: ignore
import fetch_subsequent_data # type: ignore
import get_total_past_due #type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore
import cdutils.loans.calculations # type: ignore
# import cdutils.selo # type: ignore
from _version import __version__

In [ ]:
data = fetch_data.fetch_data()

In [ ]:
wh_acctcommon = data['wh_acctcommon']

In [ ]:
effdate1 = wh_acctcommon['effdate'].iloc[-1]
effdate2 = wh_acctcommon['effdate'].iloc[-2]

In [ ]:
effdate2

In [ ]:
data2 = fetch_subsequent_data.fetch_subsequent_data(effdate1, effdate2)

In [ ]:
len(data2)

In [ ]:
curr_acctcommon, prior_acctcommon = data2['wh_acctcommon'], data2['wh_acctcommon2']
curr_acctloan, prior_acctloan = data2['wh_acctloan'], data2['wh_acctloan2']

In [ ]:
paymentstuff = data['totalpaymentsdue']

In [ ]:
paymentstuff

In [ ]:
totalpayments = get_total_past_due.isolate_total_past_due(data['totalpaymentsdue'])

curr_acctloan['acctnbr'] = curr_acctloan['acctnbr'].astype(str)
prior_acctloan['acctnbr'] = prior_acctloan['acctnbr'].astype(str)
totalpayments['acctnbr'] = totalpayments['acctnbr'].astype(str)

curr_acctloan = pd.merge(curr_acctloan, totalpayments, how='left', on='acctnbr')
curr_acctloan['totalpaymentsdue'] = curr_acctloan['totalpaymentsdue'].fillna(0)
prior_acctloan = pd.merge(prior_acctloan, totalpayments, how='left', on='acctnbr')
prior_acctloan['totalpaymentsdue'] = prior_acctloan['totalpaymentsdue'].fillna(0)

In [ ]:
curr_acctloan

In [ ]:
def test_pipeline(effdate, acctcommon, acctloan):
    acctcommon = acctcommon[acctcommon['effdate'] == effdate]
    
    acctcommon.loc[acctcommon['currmiaccttypcd'].isin(['CM15', 'CM16']), 'mjaccttypcd'] = 'CNS'

    df_for_other_pipeline = pd.merge(acctcommon, acctloan, on='acctnbr', how='left')
    
    npfm_rows = acctcommon[acctcommon['curracctstatcd'] == "NPFM"]
    
    merged1 = pd.merge(npfm_rows, acctloan, on='acctnbr', how='left')
    
    return acctloan # NEED TO FILTER ACCTLOAN

In [ ]:
x = test_pipeline(effdate1, curr_acctcommon, curr_acctloan)
x

In [ ]:
def main_pipeline(effdate, acctcommon, acctloan):
    acctcommon = acctcommon[acctcommon['effdate'] == effdate]
    
    acctcommon.loc[acctcommon['currmiaccttypcd'].isin(['CM15', 'CM16']), 'mjaccttypcd'] = 'CNS'


    acctcommon['acctnbr'] = acctcommon['acctnbr'].astype(str)
    acctloan['acctnbr'] = acctloan['acctnbr'].astype(str)
    df_for_other_pipeline = pd.merge(acctcommon, acctloan, on='acctnbr', how='left')
    
    npfm_rows = acctcommon[acctcommon['curracctstatcd'] == "NPFM"]
    
    merged1 = pd.merge(npfm_rows, acctloan, on='acctnbr', how='left')

    merged1['acctnbr'] = merged1['acctnbr'].astype(str)

    merged1['Days Past Due'] = (merged1['effdate'] - merged1['currduedate']).dt.days
    
    merged1['Days Past Due'] = np.where(merged1['Days Past Due'] < 0, 0, merged1['Days Past Due'])

    merged1 = merged1.rename(columns={
    'acctnbr': 'Account Number',
    'product': 'Product Name',
    'ownersortname': 'Customer Name',
    'loanofficer': 'Responsibility Officer',
    'bookbalance': 'Current Balance',
    'cobal': 'Charged Off',
    'riskratingcd': 'Risk',
    })

    merged1['Non Accrual'] = 'Yes'
    merged1['Net Balance'] = merged1['Current Balance'] - merged1['Charged Off']
    merged1 = merged1[merged1['Product Name'] != "Repossessed Collateral"]

    decimal_cols = ['Current Balance', 'Charged Off', 'Net Balance', 'totalpaymentsdue']
    for col in decimal_cols:
        merged1[col] = merged1[col].astype(float)
    merged1['Risk'] = merged1['Risk'].astype(str)
    merged1['Risk'] = merged1['Risk'].replace(['None', 'nan'], 'Unknown')

    merged11 = merged1[merged1['mjaccttypcd'].isin(['CML', 'MLN'])]
    


    new_row = pd.DataFrame([{
        'Current Balance': merged11['Current Balance'].sum(),
        'Charged Off': merged11['Charged Off'].sum(),
        'Net Balance': merged11['Net Balance'].sum(),
        'Non Accrual': len(merged11['Non Accrual']),
        'TagType': 'Sum'
    }])
    none_row = pd.DataFrame([{
        'TagType': None
    }])
    merged11 = pd.concat([merged11, new_row], ignore_index=True)
    merged11 = pd.concat([merged11, none_row], ignore_index=True)



    merged12 = merged1[merged1['mjaccttypcd'].isin(['CNS'])]

    # is this necessary because the filter isn't true anywhere and is doing nothing
    merged12T = merged12[merged12['Product Name'] == 'Unsecured HEAT Loans'].sort_values(by='Customer Name')
    merged12F = merged12[merged12['Product Name'] != 'Unsecured HEAT Loans']

    new_row = pd.DataFrame([{
        'Current Balance': merged12T['Current Balance'].sum(),
        'Charged Off': merged12T['Charged Off'].sum(),
        'Net Balance': merged12T['Net Balance'].sum(),
        'Non Accrual': len(merged12T['Non Accrual']),
        'TagType': 'Sum'
    }])
    
    merged12 = pd.concat([merged12T, new_row], ignore_index=True)
    merged12 = pd.concat([merged12, none_row], ignore_index=True)

    new_row = pd.DataFrame([{
        'Current Balance': merged12F['Current Balance'].sum(),
        'Charged Off': merged12F['Charged Off'].sum(),
        'Net Balance': merged12F['Net Balance'].sum(),
        'Non Accrual': len(merged12F['Non Accrual']),
        'TagType': 'Sum'
    }])

    merged13 = pd.concat([merged12F.sort_values(by='Customer Name'), new_row], ignore_index=True)
    merged13 = pd.concat([merged13, none_row], ignore_index=True)


    merged14 = merged1[merged1['mjaccttypcd'].isin(['MTG'])]

    new_row = pd.DataFrame([{
        'Current Balance': merged14['Current Balance'].sum(),
        'Charged Off': merged14['Charged Off'].sum(),
        'Net Balance': merged14['Net Balance'].sum(),
        'Non Accrual': len(merged14['Non Accrual']),
        'TagType': 'Sum'
    }])

    merged14 = pd.concat([merged14.sort_values(by='Customer Name'), new_row], ignore_index=True)
    merged14 = pd.concat([merged14, none_row], ignore_index=True)

    if not merged12['Account Number'].isna().all():
        merged_data = pd.concat([merged11, merged12, merged13, merged14], ignore_index=True)
    else:
        merged_data = pd.concat([merged11, merged13, merged14], ignore_index=True)

    merged_data = merged_data[['Product Name', 'Account Number', 'Customer Name', 'Responsibility Officer', 'Days Past Due', 'Current Balance', 'Charged Off', 'Net Balance', 'Risk', 'Non Accrual', 'totalpaymentsdue', 'currduedate', 'TagType']]
    merged_data = merged_data.rename(columns={
    'totalpaymentsdue': 'Total Amount Due',
    'currduedate': 'Next Payment Due Date'
    })


    return merged_data, merged1, df_for_other_pipeline

In [ ]:
current_report, current_report_raw, df_for_other_pipeline = main_pipeline(effdate1, curr_acctcommon, curr_acctloan)


In [ ]:
current_report

In [ ]:
totalpayments = get_total_past_due.isolate_total_past_due(data['totalpaymentsdue'])

In [ ]:
current_report

In [ ]:
current_report['Account Number'] = current_report['Account Number'].astype(str)
totalpayments['Account Number'] = totalpayments['Account Number'].astype(str)
current_report_with_new_amt_due = pd.merge(current_report, totalpayments, on='Account Number', how='left')

In [ ]:
current_report_with_new_amt_due

In [ ]:
current_report, current_report_raw, df_for_other_pipeline = main_pipeline(effdate1, curr_acctcommon, curr_acctloan)
prior_month_report, x, y = main_pipeline(effdate2, prior_acctcommon, prior_acctloan)

In [ ]:
current_report.info()

In [ ]:
  #apply(lambda x: Decimal(str(x)))

In [ ]:
current_report_raw.info()

In [ ]:
current_report.info()

In [ ]:
net_balance_sum = prior_month_report['Net Balance'].sum() / 2 # have to divide by 2 because the sum is including total rows
count_distinct_accounts = prior_month_report['Account Number'].dropna().nunique()
summary_df = pd.DataFrame({
    'Net Balance': [net_balance_sum],
    'Count': [count_distinct_accounts]
})
old_sum = summary_df

In [ ]:
prior_month_right = pd.merge(current_report_raw, prior_month_report, on='Account Number', how='right', suffixes=(None, '_right'))

df_for_other_pipeline = df_for_other_pipeline.rename(columns={
    'acctnbr': 'Account Number',
    })
df_for_other_pipeline['Account Number'] = df_for_other_pipeline['Account Number'].astype(str)
prior_month_right['Account Number'] = prior_month_right['Account Number'].astype(str)

df3 = pd.merge(df_for_other_pipeline, prior_month_right, on='Account Number', how='inner')
df3 = df3[df3['product'] == "Repossessed Collateral"]

In [ ]:
current_report

In [ ]:
new_additions = current_report[~current_report['Account Number'].isin(prior_month_report['Account Number'])] # Reconciliation section for new Non-Accrual loans
new_additions

In [ ]:
def isolate_total_past_due(df):
    df['totaldue'] = (
        df['principaldue'] +
        df['interestdue'] + 
        df['escrowdue'] +
        df['latechargedue'] +
        df['notedue'] +
        df['miscdue']
    )
    
    # Create delinquent date (due date + 15 days)
    df['delinquentdate'] = df['duedate']
    
    # Exclude payments billed that are not past due & drop helper field
    df = df[df['delinquentdate'] <= df['rundate']]
    df = df.drop(columns=['delinquentdate'])
    
    # Group by acctnbr, sum totaldue and show earliest due date
    aggregated_df = df.groupby('acctnbr').agg({
        'totaldue':'sum',
        'duedate': 'min'
    }).reset_index()
    
    aggregated_df = aggregated_df.rename(columns={
    'acctnbr': 'Account Number',
    })

    aggregated_df = aggregated_df[['Account Number', 'totaldue']]

    return aggregated_df

totalpayments = isolate_total_past_due(data['totalpaymentsdue'])

In [ ]:
# right side only
new_additions = current_report[~current_report['Account Number'].isin(prior_month_report['Account Number'])] # Reconciliation section for new Non-Accrual loans
new_additions = new_additions.drop(columns=['Current Balance', 'Charged Off'])
new_additions.insert(6, 'Total Change', new_additions['Net Balance'])
new_additions.insert(7, 'COUNT', 1)

title_row = pd.DataFrame([{
        'Customer Name': 'ADDITIONS',
        'TagType': 'Title'
    }])
new_additions = pd.concat([title_row, new_additions], ignore_index=True)
new_additions = new_additions[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]
new_additions

In [ ]:
# the rest
left_only = prior_month_report[~prior_month_report['Account Number'].isin(current_report['Account Number'])]
left_only = left_only.drop(columns=['Current Balance', 'Charged Off'])
left_only.insert(6, 'Total Change', left_only['Net Balance'] * -1)
left_only.insert(7, 'COUNT', -1)
# closed
removed = left_only[~left_only['Account Number'].isin(df3['Account Number'])] # Reconciliation section for removed from NPFM
title_row = pd.DataFrame([{
        'Customer Name': 'REMOVED',
        'TagType': 'Title'
    }])
removed = pd.concat([title_row, removed], ignore_index=True)
removed = removed[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]
removed
# removed total amount due is zero?

In [ ]:
# reposessed collateral
repo = left_only[left_only['Account Number'].isin(df3['Account Number'])].copy() # Previously on NonAccrual and now has Repo Collateral as property type

repo.loc[:, 'Product Name'] = 'Repossessed Collateral'
title_row = pd.DataFrame([{
        'Customer Name': 'INTO OTHER REPO COLLATERAL',
        'TagType': 'Title'
    }])
repo = pd.concat([title_row, repo], ignore_index=True)
repo['Total Amount Due'] = repo['Total Amount Due'].astype(float)
repo = repo[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]

repo
# repo total amount removed is zero?

In [ ]:
# right side only
new_additions = current_report[~current_report['Account Number'].isin(prior_month_report['Account Number'])] # Reconciliation section for new Non-Accrual loans
new_additions = new_additions.drop(columns=['Current Balance', 'Charged Off'])
new_additions.insert(6, 'Total Change', new_additions['Net Balance'])
new_additions.insert(7, 'COUNT', 1)

title_row = pd.DataFrame([{
        'Customer Name': 'ADDITIONS',
        'TagType': 'Title'
    }])
new_additions = pd.concat([title_row, new_additions], ignore_index=True)
new_additions = new_additions[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]


# the rest
left_only = prior_month_report[~prior_month_report['Account Number'].isin(current_report['Account Number'])]
left_only = left_only.drop(columns=['Current Balance', 'Charged Off'])
left_only.insert(6, 'Total Change', left_only['Net Balance'] * -1)
left_only.insert(7, 'COUNT', -1)
# closed
removed = left_only[~left_only['Account Number'].isin(df3['Account Number'])] # Reconciliation section for removed from NPFM
title_row = pd.DataFrame([{
        'Customer Name': 'REMOVED',
        'TagType': 'Title'
    }])
removed = pd.concat([title_row, removed], ignore_index=True)
removed = removed[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]

# reposessed collateral
repo = left_only[left_only['Account Number'].isin(df3['Account Number'])].copy() # Previously on NonAccrual and now has Repo Collateral as property type
repo['Account Number'] = repo['Account Number'].astype(str)
totalpayments['Account Number'] = totalpayments['Account Number'].astype(str)
repo = pd.merge(repo, totalpayments, on='Account Number', how='left')
repo.drop('Total Amount Due', axis=1, inplace=True)
repo = repo.rename(columns={
    'totaldue': 'Total Amount Due',
    })

repo.loc[:, 'Product Name'] = 'Repossessed Collateral'
title_row = pd.DataFrame([{
        'Customer Name': 'INTO OTHER REPO COLLATERAL',
        'TagType': 'Title'
    }])
repo = pd.concat([title_row, repo], ignore_index=True)
repo['Total Amount Due'] = repo['Total Amount Due'].astype(float)
repo = repo[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Net Balance", "Total Change", "COUNT", "Risk", "Non Accrual", "Total Amount Due", "Next Payment Due Date", "TagType"]]


In [ ]:
repo

In [ ]:
new_additions

In [ ]:
def isolate_total_past_due(df):
    df['totaldue'] = (
        df['principaldue'] +
        df['interestdue'] + 
        df['escrowdue'] +
        df['latechargedue'] +
        df['notedue'] +
        df['miscdue']
    )
    
    # Create delinquent date (due date + 15 days)
    df['delinquentdate'] = df['duedate']
    
    # Exclude payments billed that are not past due & drop helper field
    df = df[df['delinquentdate'] <= df['rundate']]
    df = df.drop(columns=['delinquentdate'])
    
    # Group by acctnbr, sum totaldue and show earliest due date
    aggregated_df = df.groupby('acctnbr').agg({
        'totaldue':'sum',
        'duedate': 'min'
    }).reset_index()
    
    aggregated_df = aggregated_df.rename(columns={
    'acctnbr': 'Account Number',
    })

    aggregated_df = aggregated_df[['Account Number', 'totaldue']]

    return aggregated_df
    

In [ ]:
totalpayments = isolate_total_past_due(data['totalpaymentsdue'])
repo['Account Number'] = repo['Account Number'].astype(str)
totalpayments['Account Number'] = totalpayments['Account Number'].astype(str)
repo = pd.merge(repo, totalpayments, on='Account Number', how='left')

In [ ]:
repo.drop('Total Amount Due', axis=1, inplace=True)
column_to_move = repo.pop('totaldue')
repo.insert(10, 'totaldue', column_to_move)
repo

In [ ]:
decimal_cols = ['Net Balance', 'Total Change', 'Total Amount Due']

In [ ]:
removed.info()

In [ ]:
current_report

In [ ]:
""" 
Non Accrual Report

This is an item for the Resolution Committee Package. This increases
operational efficiency by automating a task done by the Loan Review team.

This report tracks all non-performing loans &amp; reconciles month over month
changes.
"""

# Environment
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import time
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import Font, Border, Side, Alignment, NamedStyle
from openpyxl.styles.numbers import NumberFormat
import os

start_time = time.time()


#################################
# Most recent month end &amp; prior month end
today = datetime.today()
if today.day == 1:
    most_recent_month_end = today - timedelta(days=1)
else:
    most_recent_month_end = today.replace(day=1) - timedelta(days=1)
prior_month_end =  most_recent_month_end - MonthEnd(1)
most_recent_month_end_str = most_recent_month_end.strftime("%m/%d/%y")
prior_month_end_str = prior_month_end.strftime("%m/%d/%y")


#################################
blank_df = {
    'TagType':['Blank']
}
blank_df = pd.DataFrame(blank_df)


#################################
dfs_to_union = [new_additions, blank_df, removed, blank_df, repo]

reconciliation_df = pd.concat(dfs_to_union, ignore_index=True)


#################################
current_sum = current_report_raw['Net Balance'].sum()
net_adjustments = current_sum - old_sum['Net Balance'].iloc[0]
net_adjustments


#################################
recon_sum = reconciliation_df['Total Change'].sum()


#################################
principal_payments = net_adjustments - recon_sum
# principal_payments


#################################
output_dir = r'\\00-DA1\Home\Share\\Data & Analytics Initiatives\\Project Management\\Credit_Loan_Review\\Resolution Committee Automation\\Non Accruals\\Production'
os.makedirs(output_dir, exist_ok=True)
filename = "NonAccrualTest.xlsx"
output_file = os.path.join(output_dir,filename)

print(output_file)


#################################
reconciliation_df.info()


#################################
current_report.info()


#################################
columns_to_drop = ['MJACCTTYPCD','CURRMIACCTTYPCD','CURRACCTSTATCD','EFFDATE','NEXTDUEDATE','TOTALPI','Current Balance','Charged Off','FDIC Code','Non Accrual Date','Report']
reconciliation_df = reconciliation_df.drop(columns=[col for col in columns_to_drop if col in reconciliation_df.columns])


#################################
# Making everything into a dataframe
old_me_df = {
    'Col1':['',''],
    'Col2':['',''],
    'Customer Name':['Reconciliation', prior_month_end_str],
    'Col4':['',''],
    'Col5':['',''],
    'Col6':['',''],
    'Total Change': ['', old_sum['Net Balance'].iloc[0]],
    'Count':['', old_sum['Count'].iloc[0]]
}

old_me_df = pd.DataFrame(old_me_df)

summary_df = {
    'Col1':['','',''],
    'Col2':['','',''],
    'Customer Name':['PRINCIPAL PAYMENTS', 'TOTAL ADJUSTMENTS', most_recent_month_end_str],
    'Col4':['','',''],
    'Col5':['','',''],
    'Col6':['','',''],
    'Total Change': [principal_payments, net_adjustments, current_sum],
    'Count':['', '', len(current_report_raw)],
    'Col9':['','',''],
    'Col10':['','',''],
    'Col11':['','',''],
    'Col12':['','',''],
    'TagType':['','','Upper_Sum'] # This is a formatting tag for the new total and count
}

summary_df = pd.DataFrame(summary_df)
summary_df


#################################
with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:

    # 4 - CML Reconciliation
    old_me_df.to_excel(writer, index=False, sheet_name='Sheet1', startcol=0, startrow=5, header=False)
    
    start_row = 5 + len(old_me_df) + 1
    
    reconciliation_df.to_excel(writer, index=False, sheet_name='Sheet1', startcol=0, startrow=start_row, header=True)
    
    start_row = start_row + len(reconciliation_df) + 2
    start_save = start_row
    
    summary_df.to_excel(writer, index=False, sheet_name='Sheet1', startcol=0, startrow=start_row, header=False)
    
    start_row = start_row + len(summary_df) + 2
    end_save = start_row
    
    current_report.to_excel(writer, index=False, sheet_name='Sheet1', startcol=0, startrow=start_row, header=True)
    
    
    
wb = load_workbook(output_file)
ws = wb['Sheet1']

# Dimensions
ws.column_dimensions['A'].width = 28.33
ws.column_dimensions['B'].width = 18
ws.column_dimensions['C'].width = 36
ws.column_dimensions['D'].width = 20
ws.column_dimensions['E'].width = 9
ws.column_dimensions['F'].width = 15
ws.column_dimensions['G'].width = 18
ws.column_dimensions['H'].width = 15
ws.column_dimensions['I'].width = 15
ws.column_dimensions['J'].width = 12
ws.column_dimensions['K'].width = 17
ws.column_dimensions['L'].width = 13


# Cell Font &amp; Formatting
upper_section_font = Font(size=10, name='Arial', bold=True)
bold_italics_font = Font(size=10, name='Arial', bold=True, italic=True)
subtitle_font = Font(size=10, name='Arial', bold=True)
data_font = Font(size=10, name='Arial')
wrap_alignment = Alignment(wrap_text=True, vertical='bottom', horizontal='center')
center_alignment = Alignment(horizontal='center')
title_block = Font(size=14, name='Arial', bold=True)
accounting_style = '"$"#,##0.00_);("$"#,##0.00)'
count_format = '#,##0_);(#,##0)'

# Borders
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'))
bottom_border = Border(bottom=Side(style='thin'))
double_bottom_border= Border(bottom=Side(style='double'))
side_border = Border(left=Side(style='thin'), right=Side(style='thin'))
top_border = Border(top=Side(style='thin'))


# Header
ws['A1'] = "BRISTOL COUNTY SAVINGS BANK"
ws['A1'].font = upper_section_font
ws['A2'] = "MONTHLY NON ACCRUAL LOAN LIST"
ws['A2'].font = upper_section_font
ws['A3'] = f"{most_recent_month_end_str}"
ws['A3'].font = upper_section_font

last_row = ws.max_row

# Set column B to bold                
for row in range(1, 8):
    cell = ws.cell(row=row, column=3)
    cell.font = upper_section_font
    
for row in range(start_save, end_save):
    cell = ws.cell(row=row, column=3)
    cell.font = upper_section_font

for row in range(1, last_row+1):
    cell = ws[f'E{row}']
    cell.alignment = center_alignment
    
# Check column M for 'TagType' and assign Wrap Text and centering
for row in ws.iter_rows(min_col=13, max_col=13):
    for cell in row:
        row_idx = cell.row
        if cell.value == 'TagType':
        
            for col in ws.iter_cols(min_row=row_idx, max_row=row_idx):
                for cell in col:
                    cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
        
        elif cell.value == 'Sum':
            # Bold the cells in the subtotal rows and add borders
            for col in ws.iter_cols(min_col=6, max_col=8, min_row=row_idx, max_row=row_idx):
                for cell in col:
                    cell.font = bold_italics_font
                    cell.border = top_border + double_bottom_border
            ws.cell(row=row_idx, column=10).font = bold_italics_font
            
        elif cell.value == 'Title':
            for col in ws.iter_cols(min_row=row_idx, max_row=row_idx):
                for cell in col:
                    cell.font = upper_section_font

            
# Apply Number formatting &amp; Alignment
for row in range(1, start_row+1):
    for col in ['F','G']:
        cell = ws[f'{col}{row}']
        cell.number_format = accounting_style
    cell_h = ws[f'H{row}']
    cell_h.number_format = count_format


for row in range(start_row+1, last_row+1):
    for col in ['F','G','H']:
        cell = ws[f'{col}{row}']
        cell.number_format = accounting_style
        
for row in range(1, last_row+1):
    cell = ws[f'L{row}']
    cell.number_format = 'mm/dd/yyyy'
    
for row in range(1, last_row+1):
    cell = ws[f'J{row}']
    cell.alignment = center_alignment
    
for row in range(1, start_row+1):
    cell = ws[f'H{row}']
    cell.alignment = center_alignment
    
for row in range(1, last_row+1):
    cell = ws[f'I{row}']
    cell.alignment = center_alignment
    
for row in range(1, last_row+1):
    cell = ws[f'K{row}']
    cell.number_format = accounting_style
    
# Clearing Out Columns M &amp; N
for row in range(1, last_row+1):
    for col in [13, 14]:
        cell = ws.cell(row=row, column=col)
        cell.value = None
        cell.border = Border(left=Side(style=None), right=Side(style=None), top=Side(style=None), bottom=Side(style=None))

            
# Saving
wb.save(output_file)

print(f"Report saved to {output_file}")

# A start time variable was created at the beginning to track the number of seconds this script takes to execute.
print(f"Script took {time.time() - start_time} seconds.")

In [ ]:
summary_df

In [ ]:
repo_collat

In [ ]:
right_only

In [ ]:
left_only

In [ ]:
second_pipeline_left = pd.merge(current_report_raw, data_for_second_pipeline, on='Account Number', how='left', suffixes=(None, '_right'))
second_pipeline_right = pd.merge(current_report_raw, data_for_second_pipeline, on='Account Number', how='right', suffixes=(None, '_right'))

second_pipeline_left['Total Change'] = second_pipeline_left['Net Balance']
second_pipeline_left['Count'] = 1.0

title_row = pd.DataFrame([{
        'Customer Name': 'ADDITIONS',
        'TagType': 'Title'
    }])
df1 = pd.concat([title_row, second_pipeline_left], ignore_index=True)

df1 = df1[["Product Name", "Account Number", "Customer Name", "Responsibility Officer", "Days Past Due", "Charged Off", "Net Balance", "Total Change", "TagType", "Count", "Risk", "Non Accrual", "totalpaymentsdue", "currduedate", "mjaccttypcd", "currmiaccttypcd", "curracctstatcd", "effdate", "nextduedate", "totalpi"]]


In [ ]:
df1

In [ ]:
second_pipeline_right

In [ ]:
df_for_other_pipeline = df_for_other_pipeline.rename(columns={
    'acctnbr': 'Account Number',
    })
df_for_other_pipeline['Account Number'] = df_for_other_pipeline['Account Number'].astype(str)
second_pipeline_right['Account Number'] = second_pipeline_right['Account Number'].astype(str)

df3 = pd.merge(df_for_other_pipeline, second_pipeline_right, on='Account Number', how='inner')
df3 = df3[df3['product'] == "Repossessed Collateral"]

In [ ]:
df3

In [ ]:
second_pipeline_left

In [ ]:
current_report

In [ ]:
acct_loan_data

In [ ]:
acct_common_data.loc[acct_common_data['currmiaccttypcd'].isin(['CM15', 'CM16']), 'mjaccttypcd'] = 'CNS'

In [ ]:
npfm_rows = acct_common_data[acct_common_data['curracctstatcd'] == "NPFM"]

In [ ]:
acct_loan_data

In [ ]:
merged1 = pd.merge(npfm_rows, acct_loan_data, on='acctnbr', how='left')
merged1['acctnbr'] = merged1['acctnbr'].astype(str)

In [ ]:
merged1

In [ ]:
from datetime import datetime, timedeltaQ

merged1['Days Past Due'] = (merged1['effdate'] - merged1['currduedate']).dt.days

In [ ]:
import numpy as np
merged1['Days Past Due'] = np.where(merged1['Days Past Due'] < 0, 0, merged1['Days Past Due'])

In [ ]:
merged1.columns

In [ ]:
merged1 = merged1.rename(columns={
    'acctnbr': 'Account Number',
    'product': 'Product Name',
    'ownersortname': 'Customer Name',
    'loanofficer': 'Responsibility Officer',
    'bookbalance': 'Current Balance',
    'cobal': 'Charged Off',
    'riskratingcd': 'Risk',
    })

In [ ]:
merged1

In [ ]:
merged1['Non Accrual'] = 'Yes'
merged1['Net Balance'] = merged1['Current Balance'] - merged1['Charged Off']
merged1 = merged1[merged1['Product Name'] != "Repossessed Collateral"]

In [ ]:
merged1

In [ ]:
current_report_raw = merged1

In [ ]:
merged11 = merged1[merged1['mjaccttypcd'].isin(['CML', 'MLN'])]

In [ ]:
merged11.columns

In [ ]:
new_row = pd.DataFrame([{
    'Current Balance': merged11['Current Balance'].sum(),
    'Charged Off': merged11['Charged Off'].sum(),
    'Net Balance': merged11['Net Balance'].sum(),
    'Non Accrual': len(merged11['Non Accrual']),
}])
none_row = pd.DataFrame([{
    'Current Balance': None
}])
merged11 = pd.concat([merged11, new_row], ignore_index=True)
merged11 = pd.concat([merged11, none_row], ignore_index=True)

In [ ]:
merged11

In [ ]:
merged12 = merged1[merged1['mjaccttypcd'].isin(['CNS'])]

# is this necessary because the filter isn't true anywhere and is doing nothing
merged12T = merged12[merged12['Product Name'] == 'Unsecured HEAT Loans'].sort_values(by='Customer Name')
merged12F = merged12[merged12['Product Name'] != 'Unsecured HEAT Loans']

new_row = pd.DataFrame([{
    'Current Balance': merged12T['Current Balance'].sum(),
    'Charged Off': merged12T['Charged Off'].sum(),
    'Net Balance': merged12T['Net Balance'].sum(),
    'Non Accrual': len(merged12T['Non Accrual']),
}])

merged12 = pd.concat([merged12T, new_row], ignore_index=True)
merged12 = pd.concat([merged12, none_row], ignore_index=True)

In [ ]:
merged12

In [ ]:
new_row = pd.DataFrame([{
    'Current Balance': merged12F['Current Balance'].sum(),
    'Charged Off': merged12F['Charged Off'].sum(),
    'Net Balance': merged12F['Net Balance'].sum(),
    'Non Accrual': len(merged12F['Non Accrual']),
}])

merged13 = pd.concat([merged12F.sort_values(by='Customer Name'), new_row], ignore_index=True)
merged13 = pd.concat([merged13, none_row], ignore_index=True)

In [ ]:
merged14 = merged1[merged1['mjaccttypcd'].isin(['MTG'])]

new_row = pd.DataFrame([{
    'Current Balance': merged14['Current Balance'].sum(),
    'Charged Off': merged14['Charged Off'].sum(),
    'Net Balance': merged14['Net Balance'].sum(),
    'Non Accrual': len(merged14['Non Accrual']),
}])

merged14 = pd.concat([merged14.sort_values(by='Customer Name'), new_row], ignore_index=True)
merged14 = pd.concat([merged14, none_row], ignore_index=True)

In [ ]:
merged14

In [ ]:
merged_data = pd.concat([merged11, merged12, merged13, merged14], ignore_index=True)

In [ ]:
merged_data

In [ ]:
current_report = merged_data
current_report_raw = merged1
temp = merged1